In [84]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import os

from PyPDF2 import PdfFileReader
import sys
try:
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.pdfpage import PDFPage
    from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
    from pdfminer.layout import LAParams
except:
    !pip install pdfminer.six
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.pdfpage import PDFPage
    from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
    from pdfminer.layout import LAParams
import io
from urllib.request import urlopen
import pandas as pd

import re
try:
    from tika import parser
except:
    !pip install tika
    from tika import parser
import shutil
from os import listdir
from os.path import isfile, join

# 1. pdf파일 전체 다운받기

In [52]:
def downloadURLResource(url,title, place):
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        targetFileName = title + '.pdf'
        with open(place + "{}".format(targetFileName), 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                f.write(chunk)

In [80]:
def download_doctorrock_pdf(enddate='20191231',startdate='20050101', pdf_folder='C:/Users/Funzo/Desktop/doctorrock_pdf/'):
    end = pd.to_datetime(enddate)
    start = pd.to_datetime(startdate)
    allowed_date_list = pd.date_range(start,end)
    
    title_list = []
    date_list = []
    link_list = []
    stop_signal = 0
    for page in range(1, 100):
        url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(page)
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content)

        table = soup.find('div', class_='bdLine type2').find('ul')
        for i in range(10):
            title = table.find_all('li')[i*3].find('span',class_='titlesub').text

            meeting_date = pd.to_datetime(title.split('(')[2].replace(')',''))

            if meeting_date in allowed_date_list:

                date = table.find_all('li')[i*3].find('span',class_='date').text[3:]
                
                # 더 이상 pdf형식을 지원하지 않을 경우 탈출        
                if table.find_all('li')[i*3].find('div',class_='fileGoupBox') == None:
                    print('{}페이지 {}번째 파일부터 pdf형식을 제공하지 않습니다.'.format(page,i))
                    stop_signal = 1
                    break
                    
                pdf_or_hwp = table.find_all('li')[i*3].find('div',class_='fileGoupBox').find('ul')
                if pdf_or_hwp.find_all('li')[0].find('a').text.split('.')[1][:3] == 'pdf':
                    link = pdf_or_hwp.find_all('li')[0].find('a')['href']
                else:
                    link = pdf_or_hwp.find_all('li')[1].find('a')['href']

                title_list.append(title)
                date_list.append(date)
                link_list.append(link)
            
            elif int(str(meeting_date - end).split(' days ')[0]) > 0:
                continue

            else:
                stop_signal = 1
                break
                
        if stop_signal == 1:
            break
        print(page,'페이지 완료')

    names_urls = zip(title_list, date_list, link_list)
    base_pdf = 'http://www.bok.or.kr'

    for title, date, link in names_urls:
        name = title+'_'+date
        url = base_pdf + link
        downloadURLResource(url, name, pdf_folder)
        print(name, '저장 완료')
        
    return title_list

In [78]:
data = download_doctorrock_pdf()

1 페이지 완료
2 페이지 완료
3 페이지 완료
4 페이지 완료
5 페이지 완료
6 페이지 완료
7 페이지 완료
8 페이지 완료
9 페이지 완료
10 페이지 완료
11 페이지 완료
12 페이지 완료
13 페이지 완료
14 페이지 완료
15 페이지 완료
16 페이지 완료
17 페이지 완료
18 페이지 완료
19 페이지 완료
20 페이지 완료
21 페이지 완료
22 페이지 완료
23 페이지 완료
24 페이지 완료
25 페이지 완료
26 페이지 완료
27 페이지 완료
28 페이지 완료
29 페이지 완료
30페이지 2번째 파일부터 pdf형식을 제공하지 않습니다.
금융통화위원회 의사록(2019년도 제12차)(2019.6.20.)_2019.07.09 저장 완료
금융통화위원회 의사록(2019년도 제10차)(2019.5.31.)_2019.06.18 저장 완료
금융통화위원회 의사록(2019년도 제9차)(2019.5.9)_2019.05.28 저장 완료
금융통화위원회 의사록(2019년도 제7차)(2019.4.18.)_2019.05.07 저장 완료
금융통화위원회 의사록(2019년도 제6차)(2019.3.28.)_2019.04.12 저장 완료
금융통화위원회 의사록(2019년도 제4차)(2019.2.28.)_2019.03.19 저장 완료
금융통화위원회 의사록(2019년도 제3차)(2019.2.14.)_2019.03.05 저장 완료
금융통화위원회 의사록(2019년도 제2차)(2019.1.24.)_2019.02.12 저장 완료
금융통화위원회 의사록(2019년도 제1차)(2019.1.17.)_2019.02.07 저장 완료
금융통화위원회 의사록(2018년도 제25차)(2018.12.26.) _2019.01.15 저장 완료
금융통화위원회 의사록(2018년도 제24차)(2018.12.20.) _2019.01.08 저장 완료
금융통화위원회 의사록(2018년도 제23차)(2018.12.6.)_2018.12.26 저장 완료
금융통화위원회 의사록(2018년도 제22차)(2018.11.30.)_2018.

금융통화위원회 의사록(2011년도 제29차)(2011.12.22)_2012.02.07 저장 완료
금융통화위원회 의사록(2011년도 제28차)(2011.12.16)_2012.01.31 저장 완료
금융통화위원회 의사록(2011년도 제27차)(2011.12.8)_2012.01.25 저장 완료
금융통화위원회 의사록(2011년도 제26차)(2011.11.24)_2012.01.13 저장 완료
금융통화위원회 의사록(2011년도 제25차)(2011.11.11)_2011.12.27 저장 완료
금융통화위원회 의사록(2011년도 제23차)(2011.10.13)_2011.11.29 저장 완료
금융통화위원회 의사록(2011년도 제22차)(2011.9.22)_2011.11.08 저장 완료
금융통화위원회 의사록(2011년도 제21차)(2011.9.8)_2011.10.25 저장 완료
금융통화위원회 의사록(2011년도 제19차)(2011.8.11)_2011.09.27 저장 완료
금융통화위원회 의사록(2011년도 제18차)(2011.7.21)_2011.09.06 저장 완료
금융통화위원회 의사록(2011년도 제17차)(2011.7.14)_2011.08.30 저장 완료
금융통화위원회 의사록(2011년도 제16차)(2011.6.23)_2011.08.09 저장 완료
금융통화위원회 의사록(2011년도 제15차)(2011.6.10)_2011.07.26 저장 완료
금융통화위원회 의사록(2011년도 제13차)(2011.5.13)_2011.06.28 저장 완료
금융통화위원회 의사록(2011년도 제9차)(2011.4.12)_2011.05.31 저장 완료
금융통화위원회 의사록(2011년도 제8차)(2011.3.24)_2011.05.11 저장 완료
금융통화위원회 의사록(2011년도 제7차)(2011.3.10)_2011.04.26 저장 완료
금융통화위원회 의사록(2011년도 제6차)(2011.2.24)_2011.04.12 저장 완료
금융통화위원회 의사록(2011년도 제5차)(2011.2.11)_2011.03.29 

# 2. text파일로 저장하기

In [85]:
def pdf2txt (source_folder='C:/Users/Funzo/Desktop/doctorrock_pdf/', output_folder='C:/Users/Funzo/Desktop/doctorrock_txt/') :
    #에러가 발생한 파일이름 저장준비
    error_list = []
    pdf_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    for pdf in pdf_files :
        pdf_filepath = source_folder + pdf
        pdf_tmp_filepath = output_folder + 'tmp.pdf'
        # pdf 파일을 text로 변환

        shutil.copyfile(pdf_filepath, pdf_tmp_filepath)
        
        #주로 에러가 발생하는 코드
        try:
            parsedPDF = re.sub('\n', '', parser.from_file(pdf_tmp_filepath)["content"])
        except:
            print(pdf)
            error_list.append(pdf)
            pass
        
        ouput_filepath = (output_folder + pdf).replace('.pdf', '.txt')
        with open(ouput_filepath, 'w',encoding='utf-8' ) as f :
            print(ouput_filepath)
            f.write(parsedPDF)
            f.close()    

    return error_list

In [86]:
errors = pdf2txt()

C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2005년도 제12차)(2005.6.9)_2005.07.26.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2005년도 제13차)(2005.6.23)_2005.08.09.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2005년도 제14차)(2005.7.7)_2005.08.23.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2005년도 제17차)(2005.8.11)_2005.09.27.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2005년도 제19차)(2005.9.8)_2005.10.25.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2005년도 제20차)(2005.9.22)_2005.11.08.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2005년도 제21차)(2005.10.11)_2005.11.29.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2005년도 제24차)(2005.11.10)_2005.12.27.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2005년도 제26차)(2005.12.8)_2006.01.24.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2005년도 제27차)(2005.12.22)_2006.02.07.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2006년도 제10차)(2006.5.11)_2006.06.27.txt
C:/Users/Funzo/Deskto

C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2009년도 제2차)(2009.1.9)_2009.02.24.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2009년도 제3차)(2009.1.22)_2009.03.10.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2009년도 제4차)(2009.2.12)_2009.03.31.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2009년도 제5차)(2009.2.25)_2009.04.14.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2009년도 제7차)(2009.3.12)_2009.04.28.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2009년도 제8차)(2009.3.26)_2009.05.12.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2009년도 제9차)(2009.4.9)_2009.05.26.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2010년도 제10차)(2010.5.12)_2010.06.29.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2010년도 제12차)(2010.6.10)_2010.07.27.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2010년도 제13차)(2010.6.24)_2010.08.10.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2010년도 제14차)(2010.7.9)_2010.08.24.txt
C:/Users/Funzo/Desktop/doctorro

C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제15차)(2014.8.14.)_2014.09.02.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제17차)(2014.9.12.)_2014.09.30.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제18차)(2014.9.25.)_2014.10.14.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제19차)(2014.10.15.)_2014.11.04.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제1차)(2014.1.9)_2014.01.28.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제20차)(2014.10.23.)_2014.11.11.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제21차)(2014.11.13.)_2014.12.02.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제23차)(2014.12.11.)_2014.12.30.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제24차)(2014.12.24.)_2015.01.13.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제3차)(2014.2.13.)_2014.03.04.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2014년도 제5차)(2014.3.13.)_2014.04.01.txt
C:/Users/F

C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2018년도 제4차)(2018.2.27.)_2018.03.20.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2018년도 제5차)(2018.3.8.)_2018.03.27.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2018년도 제6차)(2018.3.29.)_2018.04.17.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2018년도 제7차)(2018.4.12.)_2018.05.02.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2018년도 제8차)(2018.4.26.)_2018.05.15.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2019년도 제10차)(2019.5.31.)_2019.06.18.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2019년도 제12차)(2019.6.20.)_2019.07.09.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2019년도 제1차)(2019.1.17.)_2019.02.07.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2019년도 제2차)(2019.1.24.)_2019.02.12.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2019년도 제3차)(2019.2.14.)_2019.03.05.txt
C:/Users/Funzo/Desktop/doctorrock_txt/금융통화위원회 의사록(2019년도 제4차)(2019.2.28.)_2019.03.19.txt
C:/Users/Funzo/Deskt

In [87]:
errors

[]

In [92]:
int(str(pd.to_datetime('20190101')-pd.to_datetime('20181212')).split(' days ')[0])

20